## RAW DATA CHECK (Bronze Source)

In [0]:
from pyspark.sql.functions import current_timestamp

bronze_df = (
    spark.table("default.movies")
    .withColumn("ingestion_time", current_timestamp())
)

bronze_df.show(5)
bronze_df.printSchema()


+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|               title| industry|release_year|       imdb_rating|              studio|  budget|             revenue|     unit|currency|language|      ingestion_time|
+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|     Pather Panchali|Bollywood|        1955|8.3000000000000007|Government of Wes...|70000.00|100000.0000000000...|Thousands|     INR| Bengali|2026-01-14 07:45:...|
|Doctor Strange in...|Hollywood|        2022|                 7|      Marvel Studios|  200.00|  954.80000000000000| Millions|     USD| English|2026-01-14 07:45:...|
|Thor: The Dark Wo...|Hollywood|        2013|               6.8|      Marvel Studios|  165.00|  644.79999999999995| Millions|     USD| English|2026-01-14 07:45:...|
|     Thor

## BRONZE LAYER – Raw Ingestion

In [0]:
from pyspark.sql.functions import current_timestamp

bronze_df = (
    spark.table("default.movies")
    .withColumn("ingestion_time", current_timestamp())
)

bronze_df.show(5)
bronze_df.printSchema()


+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|               title| industry|release_year|       imdb_rating|              studio|  budget|             revenue|     unit|currency|language|      ingestion_time|
+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|     Pather Panchali|Bollywood|        1955|8.3000000000000007|Government of Wes...|70000.00|100000.0000000000...|Thousands|     INR| Bengali|2026-01-14 07:46:...|
|Doctor Strange in...|Hollywood|        2022|                 7|      Marvel Studios|  200.00|  954.80000000000000| Millions|     USD| English|2026-01-14 07:46:...|
|Thor: The Dark Wo...|Hollywood|        2013|               6.8|      Marvel Studios|  165.00|  644.79999999999995| Millions|     USD| English|2026-01-14 07:46:...|
|     Thor

In [0]:
bronze_df.write.mode("overwrite").saveAsTable("default.bronze_movies")


## SILVER LAYER – Cleaning & Validation

In [0]:
from pyspark.sql.functions import col

silver_df = (
    bronze_df
    .filter(col("title").isNotNull())
    .filter(col("industry").isNotNull())
    .filter(col("studio").isNotNull())
    .filter(col("imdb_rating").isNotNull())
    .filter(col("budget").isNotNull() & (col("budget") >= 0))
    .filter(col("revenue").isNotNull() & (col("revenue") >= 0))
    .dropDuplicates()
)

silver_df.show(5)


+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|               title| industry|release_year|       imdb_rating|              studio|  budget|             revenue|     unit|currency|language|      ingestion_time|
+--------------------+---------+------------+------------------+--------------------+--------+--------------------+---------+--------+--------+--------------------+
|     Pather Panchali|Bollywood|        1955|8.3000000000000007|Government of Wes...|70000.00|100000.0000000000...|Thousands|     INR| Bengali|2026-01-14 07:46:...|
|Doctor Strange in...|Hollywood|        2022|                 7|      Marvel Studios|  200.00|  954.80000000000000| Millions|     USD| English|2026-01-14 07:46:...|
|Thor: The Dark Wo...|Hollywood|        2013|               6.8|      Marvel Studios|  165.00|  644.79999999999995| Millions|     USD| English|2026-01-14 07:46:...|
|     Thor

In [0]:
silver_df.write.mode("overwrite").saveAsTable("default.silver_movies")


## GOLD LAYER (Business Aggregates)

In [0]:
from pyspark.sql.functions import count, sum, avg

gold_studio_df = (
    silver_df
    .groupBy("studio")
    .agg(
        count("*").alias("movie_count"),
        sum("budget").alias("total_budget"),
        sum("revenue").alias("total_revenue"),
        avg("imdb_rating_clean").alias("avg_imdb_rating")
    )
)

gold_studio_df.show(10)


+--------------------+-----------+------------+--------------------+---------------+
|              studio|movie_count|total_budget|       total_revenue|avg_imdb_rating|
+--------------------+-----------+------------+--------------------+---------------+
|Government of Wes...|          1|    70000.00|100000.0000000000...|            8.3|
|      Marvel Studios|          8|     1988.70| 9054.59999999999995|            7.5|
|Castle Rock Enter...|          1|       25.00|   73.30000000000000|            9.3|
|Warner Bros. Pict...|          1|      165.00|  701.80000000000000|            8.6|
|   Columbia Pictures|          1|       55.00|  307.10000000000002|            8.0|
|Universal Pictures  |          1|      103.00|  460.50000000000000|            8.5|
|  Paramount Pictures|          2|      207.20| 2493.00000000000000|           8.55|
|       Liberty Films|          1|        3.18|    3.30000000000000|            8.6|
|    20th Century Fox|          1|      237.00| 2847.000000000000